# HW 2 - Carlos Alvarado

In [1]:
import sys
sys.path.append('../source')

from helper_functions import read_data, discretize, make_dummies

data = read_data('data/credit-data.csv')

In [ ]:
make_dummies(data, 'DebtRatio')

<class 'numpy.float64'> 0.802982129
<class 'numpy.float64'> 0.121876201
<class 'numpy.float64'> 0.085113375
<class 'numpy.float64'> 0.036049682
<class 'numpy.float64'> 0.024925695
<class 'numpy.float64'> 0.375606969
<class 'numpy.float64'> 5710.0
<class 'numpy.float64'> 0.209940017
<class 'numpy.float64'> 46.0
<class 'numpy.float64'> 0.606290901
<class 'numpy.float64'> 0.30947621
<class 'numpy.float64'> 0.53152876
<class 'numpy.float64'> 0.298354075
<class 'numpy.float64'> 0.382964747
<class 'numpy.float64'> 477.0
<class 'numpy.float64'> 0.209891754
<class 'numpy.float64'> 2058.0
<class 'numpy.float64'> 0.18827406
<class 'numpy.float64'> 0.527887839
<class 'numpy.float64'> 0.065868263
<class 'numpy.float64'> 0.430046338
<class 'numpy.float64'> 0.475841386
<class 'numpy.float64'> 0.241103559
<class 'numpy.float64'> 0.085512189
<class 'numpy.float64'> 0.241621845
<class 'numpy.float64'> 1.595253368
<class 'numpy.float64'> 0.097672186
<class 'numpy.float64'> 0.042383047
<class 'numpy.floa

In [7]:
data[data['age']>30]

,PersonID,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,zipcode,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,...,age_0,age_109,zipcode_60644,zipcode_60637,zipcode_60601,zipcode_60625,zipcode_60629,zipcode_60804,zipcode_60657,zipcode_60618
0,1,1,0.766127,45,60644,2,0.802982,9120.0,13,0,...,False,False,True,False,False,False,False,False,False,False
1,2,0,0.957151,40,60637,0,0.121876,2600.0,4,0,...,False,False,False,True,False,False,False,False,False,False
2,3,0,0.658180,38,60601,1,0.085113,3042.0,2,1,...,False,False,False,False,True,False,False,False,False,False
4,5,0,0.907239,49,60625,1,0.024926,63588.0,7,0,...,False,False,False,False,False,True,False,False,False,False
5,6,0,0.213179,74,60629,0,0.375607,3500.0,3,0,...,False,False,False,False,False,False,True,False,False,False
6,7,0,0.305682,57,60637,0,5710.000000,NaN,8,0,...,False,False,False,True,False,False,False,False,False,False
7,8,0,0.754464,39,60625,0,0.209940,3500.0,8,0,...,False,False,False,False,False,True,False,False,False,False
9,10,0,0.189169,57,60629,0,0.606291,23684.0,9,0,...,False,False,False,False,False,False,True,False,False,False
11,12,0,0.018798,51,60804,0,0.531529,6501.0,7,0,...,False,False,False,False,False,False,False,True,False,False
12,13,0,0.010352,46,60644,0,0.298354,12454.0,13,0,...,False,False,True,False,False,False,False,False,False,False


In [ ]:
discretize(data, "age", 10, cut_type = 'logspace')
data.columns

In [ ]:
#quantile
data.age.describe()

In [ ]:
#uniform
data.DebtRatio_cat.value_counts()

In [ ]:
#linspace
data.DebtRatio_cat.value_counts()

In [ ]:
#logspace
data.DebtRatio_cat.value_counts()